In [1]:
!pip install langchain transformers langchain-community


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain.utilities import SQLDatabase
from IPython.display import display

# SQLCoder model setup
model_name = "defog/llama-3-sqlcoder-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Database setup
SQLITE_URI = "sqlite:///Material.db"
db = SQLDatabase.from_uri(SQLITE_URI, include_tables=['materials'], sample_rows_in_table_info=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [3]:
def generate_text(prompt: str, max_length: int = 512) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_sql(user_question: str, schema: str) -> str:
    prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>
Generate a SQL query to answer this question: `{user_question}`
Use only the columns provided in the DDL statement.
DDL statements:
{schema}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
The following SQL query best answers the question `{user_question}`:
```sql
"""
    generated_sql = generate_text(prompt)
    sql_start = generated_sql.find("```sql") + 6
    sql_end = generated_sql.find("```", sql_start)
    return generated_sql[sql_start:sql_end].strip()

def execute_query(query: str) -> str:
    try:
        result = db.run(query)
        print("Query Result:")
        display(result)
        return result
    except Exception as e:
        error_message = f"An error occurred while executing the query: {str(e)}"
        print(error_message)
        return error_message

def generate_response(user_question: str, query_result: str) -> str:
    prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>
As a professional representative of a manufacturing and logistics company,
provide a clear and concise response to the following user query based on the database result.
Make sure the response is easy for a regular user to understand.

User Query: {user_question}

Database Result: {query_result}

Please provide a detailed response based on this information:
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
    return generate_text(prompt, max_length=1024)

def main(user_question: str, schema: str):
    print(f"User Question: {user_question}\n")

    sql_query = generate_sql(user_question, schema)
    print("Generated SQL Query:")
    print(sql_query)
    print()

    query_result = execute_query(sql_query)
    print()

    response = generate_response(user_question, query_result)
    print("Generated Response:")
    print(response)

# Schema
schema = """
CREATE TABLE materials (
    "index" INTEGER,
    "Unnamed: 0" INTEGER,
    "BOMcat" TEXT,
    "BOM" INTEGER,
    "Unnamed: 2" TEXT,
    "Material" TEXT,
    "Material Description" TEXT,
    "Plant" INTEGER,
    "BOM Usage" INTEGER,
    "Alternative" INTEGER,
    "Creation Date" TIMESTAMP,
    "User" TEXT,
    "Component" TEXT,
    "Material Description.1" TEXT,
    "Class" TEXT,
    "ICt" TEXT,
    "Item" TEXT,
    "Quantity" REAL,
    "Un" TEXT,
    "C.scrap" REAL,
    "Prod." TEXT,
    "Costing" TEXT
)
"""


In [6]:
# Example usage
user_question = "What are the components in the product with Material code: OFSO2SBG?"
main(user_question, schema)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


User Question: What are the components in the product with Material code: OFSO2SBG?

Generated SQL Query:
SELECT m.component FROM materials m WHERE m.material = 'OFSO2SBG'

Query Result:


"[(None,), (None,), (None,), ('SO310.LEGN',), ('SO310S2A140.WCH',), ('ACCCHD45110100160',), ('FOFOA7500900310-1',), ('FOFOA7500900310-2',), ('FOFOP200012001028',), ('FOFOS40702003712_0',), ('TRFAX340040041412',), ('UAUAK330120000101',), ('UAUAK33012000100',), ('UAUAW330110001171',), ('UAUAY3300600006412',), ('UAUAY3300900009412',), ('UAUTP33012000170',), ('PKVEB40900205320',), ('PKVEB40900200310',), ('PKPKRW40907101511',), ('SWSWA520040090',), ('CH111.MT35',), ('EDEDS405017002416',), ('UAUTP3301500013412',)]"

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generated Response:
user
As a professional representative of a manufacturing and logistics company, 
provide a clear and concise response to the following user query based on the database result.
Make sure the response is easy for a regular user to understand.

User Query: What are the components in the product with Material code: OFSO2SBG?

Database Result: [(None,), (None,), (None,), ('SO310.LEGN',), ('SO310S2A140.WCH',), ('ACCCHD45110100160',), ('FOFOA7500900310-1',), ('FOFOA7500900310-2',), ('FOFOP200012001028',), ('FOFOS40702003712_0',), ('TRFAX340040041412',), ('UAUAK330120000101',), ('UAUAK33012000100',), ('UAUAW330110001171',), ('UAUAY3300600006412',), ('UAUAY3300900009412',), ('UAUTP33012000170',), ('PKVEB40900205320',), ('PKVEB40900200310',), ('PKPKRW40907101511',), ('SWSWA520040090',), ('CH111.MT35',), ('EDEDS405017002416',), ('UAUTP3301500013412',)]

Please provide a detailed response based on this information:
assistant
Based on the provided database result, the component